In [49]:
import csv
import re

In [23]:
from data.models import *

## Carregando nomes dos indicadores

In [76]:
dicionario = [_.decode('utf-8').strip().split('|') for _ in open('etl/dicionario.txt').read().split('\n') if _.strip()]
dicionario.pop()
codigos = {var[1]:var for var in dicionario}

# Extraindo periodo
periodos = set()
for tema, cod, nome in dicionario:
    nome = nome.strip()
    m = re.search(r'((?<=\s)\d{4}$|(?<=\()\d{4}(?=\))|(?<=\()\d{4}-\d{4}(?=\))|(?<=Dezembro/)\d{4}(?=\)))', nome)
    if not m: 
        print repr(nome)
    else:
        periodos.add(m.group(1))
        
periodos

u'Taxa m\xe9dia geom\xe9trica de crescimento anual da popula\xe7\xe3o 2000/2010 (%)'
u'Valor do rendimento nominal m\xe9dio mensal do trabalho principal das pessoas de 10 anos ou mais de idade, ocupadas na semana de refer\xeancia, com rendimento do trabalho principal (Reais)'
u'Taxa de mortalidade por doen\xe7as cerebrovasculares (em 100 mil habitantes) (20008-2010)'
u'Total de M\xe9dicos em atividade em 12.05.2014 (Mais M\xe9dicos)'
u'M\xe9dicos do programa Mais M\xe9dicos por mil habitantes (Maio/2014)'
u'% de popula\xe7\xe3o benefici\xe1ria de planos de sa\xfade (Setembro/2015)'


{u'2000',
 u'2008',
 u'2008-2010',
 u'2009',
 u'2010',
 u'2010-2012',
 u'2011',
 u'2011-2013',
 u'2012',
 u'2012-2014',
 u'2013',
 u'2014',
 u'2015',
 u'2016',
 u'2017',
 u'2018',
 u'2019',
 u'2020',
 u'2021',
 u'2022',
 u'2023',
 u'2024',
 u'2025',
 u'2026',
 u'2027',
 u'2028',
 u'2029',
 u'2030'}

## Carregando CSV com os dados

In [26]:
reader = csv.reader(open('etl/Base-Projeto-BID-Indicadores-Regioes-de-Saude-SP.csv'), delimiter=';')
headers = next(reader)

## Cabeçalhos que não existem na tabela de códigos de indicador

In [27]:
for header in headers:
    if header not in codigos:
        print header

REGSAUDCOD_BID
REGSAUDNOME_BID


## Códigos que não existem no cabeçalho

In [48]:
for codigo in codigos:
    if codigo not in headers:
        print codigo

## Carregar dados

In [33]:
# Todas as linhas
lines = list(reader)

In [37]:
# Inspecionando localidades
localidades = set(tuple(line[:2]) for line in lines)
localidades

{('35011', 'Alto do Tiet\xea'),
 ('35012', 'Franco da Rocha'),
 ('35013', 'Mananciais'),
 ('35014', 'Rota dos Bandeirantes'),
 ('35015', 'Grande ABC'),
 ('35016', 'S\xe3o Paulo'),
 ('35021', 'Central do DRS II'),
 ('35022', 'Lagos do DRS II'),
 ('35023', 'Cons\xf3rcios do DRS II'),
 ('35031', 'Central do DRS III'),
 ('35032', 'Centro Oeste do DRS III'),
 ('35033', 'Norte do DRS III'),
 ('35034', 'Cora\xe7\xe3o do DRS III'),
 ('35041', 'Baixada Santista'),
 ('35051', 'Norte - Barretos'),
 ('35052', 'Sul - Barretos'),
 ('35061', 'Vale do Jurumirim'),
 ('35062', 'Bauru'),
 ('35063', 'Polo Cuesta'),
 ('35064', 'Ja\xfa'),
 ('35065', 'Lins'),
 ('35071', 'Bragan\xe7a'),
 ('35072', 'Regi\xe3o Metropolitana de Campinas'),
 ('35073', 'Jundia\xed'),
 ('35074', 'Circuito das \xc1guas'),
 ('35081', 'Tr\xeas Colinas'),
 ('35082', 'Alta Anhanguera'),
 ('35083', 'Alta Mogiana'),
 ('35091', 'Adamantina'),
 ('35092', 'Assis'),
 ('35093', 'Mar\xedlia'),
 ('35094', 'Ourinhos'),
 ('35095', 'Tup\xe3'),
 ('3

In [35]:
len(localidades)

63

In [142]:
regioes = {}
for cod, nome in localidades:
    nome = nome.decode("iso-8859-1")
    try:
        localidade = Localidade.objects.get(nome=nome, codigo=cod, tipo='regiao')
    except Localidade.DoesNotExist:
        localidade = Localidade.objects.create(nome=nome, codigo=cod, tipo='regiao')
    else:
        print 'localidade ja existe: ', nome
    regioes[cod] = localidade

localidade ja existe:  Rio Claro
localidade ja existe:  Marília
localidade ja existe:  Polo Cuesta
localidade ja existe:  Catanduva
localidade ja existe:  Baixada Santista
localidade ja existe:  José Bonifácio
localidade ja existe:  Fernandópolis
localidade ja existe:  Jales
localidade ja existe:  Limeira
localidade ja existe:  Rio Pardo
localidade ja existe:  Santa Fé do Sul
localidade ja existe:  Norte do DRS III
localidade ja existe:  Alta Sorocabana
localidade ja existe:  Sul - Barretos
localidade ja existe:  Itapetininga
localidade ja existe:  Adamantina
localidade ja existe:  Tupã
localidade ja existe:  Região Metropolitana de Campinas
localidade ja existe:  Extremo Oeste Paulista
localidade ja existe:  Coração do DRS III
localidade ja existe:  Vale do Ribeira
localidade ja existe:  Ourinhos
localidade ja existe:  Sorocaba
localidade ja existe:  Consórcios do DRS II
localidade ja existe:  Mantiqueira
localidade ja existe:  Jaú
localidade ja existe:  Assis
localidade ja existe:  V

In [75]:
def get_periodo(codigo):
    tema, _, nome = codigos[codigo]
    nome = nome.strip()
    m = re.search(r'((?<=\s)\d{4}$|(?<=\()\d{4}(?=\))|(?<=\()\d{4}-\d{4}(?=\))|(?<=Dezembro/)\d{4}(?=\)))', nome)
    if not m:
        print 'periodo nao encontrado', nome
        return None
    return m.group(1)

In [79]:
# Temas
temas = {tema:None for tema, codigo, nome in dicionario}

In [82]:
for i, nome in enumerate(temas):
    try:
        temas[nome] = Tema.objects.get(nome=nome)
    except Tema.DoesNotExist:
        temas[nome] = Tema.objects.create(nome=nome, ordem=i+1)

In [101]:
na = Fonte.objects.get(pk=16)

In [149]:
# Indicadores
indicador_periodos = {}
codigo_periodo = {}
codigo_variavel = {}
for t, c, n in dicionario:
    tema = temas[t]
    m = re.search(r'(.+?) (\d{4})$', n)
    if not m:
        m = re.search(r'(.+?) \((\d{4})\)$', n)
        if not m:
            m = re.search(r'(.+?) \((\d{4})[-/]\d{4}\)$', n)
    if m:
        nome, ano = m.groups()
        nome = nome.strip('-').strip()
        indicador_periodos.setdefault((tema, nome), [set(), set()])[0].add(ano)
        indicador_periodos[(tema, nome)][1].add(c)
        codigo_periodo[c] = ano
                                      
indicadores = {}
for (tema, nome), (periodos, codigos) in indicador_periodos.items():
    for codigo in codigos:
        print tema.nome, nome, codigo
        try:
            indicadores[codigo] = tema.indicador_set.get(nome=nome)
        except Indicador.DoesNotExist:
            indicadores[codigo] = Indicador.objects.create(
                tema=tema,
                nome=nome,
                fonte=na,
                periodo=','.join(sorted(periodos))
            )


ESCOLARIDADE % de pessoas de 10 anos e mais com pelo menos ensino médio P_EMCOMP_10
ESTRUTURA Total de unidades de saúde de atenção básica UNSAUDE_12
MORBIDADE Taxa de incidência de Malária (em 100 mil habitantes) TXINCMALAR_1214
MORBIDADE Taxa de incidência de Malária (em 100 mil habitantes) TXINCMALAR_0810
ESTRUTURA Concluintes de medicina por 100 mil habitantes CONCL_MEDHAB_13
ESTRUTURA Concluintes de medicina por 100 mil habitantes CONCL_MEDHAB_14
ESTRUTURA Número de concluintes - cursos de enfermagem CONCL_ENFERM_14
ESTRUTURA Número de concluintes - cursos de enfermagem CONCL_ENFERM_13
CONDIÇÕES DE SAÚDE Taxa de mortalidade infantil - menores de 1 ano (em 1.000 nascidos vivos) TXMORTINF_1113
CONDIÇÕES DE SAÚDE Taxa de mortalidade infantil - menores de 1 ano (em 1.000 nascidos vivos) TXMORTINF_0810
VULNERABILIDADE SOCIAL % de pessoas em domicílios vulneráveis à pobreza e dependentes de idosos DEPIDOSO_10
FINANCIAMENTO Recursos de Transferências SUS por habitante RTRSUS_HAB_10
FINAN

In [152]:
indicadores

{u'AGUA_10': <Indicador: Percentual de pessoas residentes em domicílios com acesso à abastecimento de água>,
 u'COBPOP_EAB_08': <Indicador: Cobertura populacional estimada pelas equipes de Atenção Básica>,
 u'COBPOP_EAB_09': <Indicador: Cobertura populacional estimada pelas equipes de Atenção Básica>,
 u'COBPOP_EAB_10': <Indicador: Cobertura populacional estimada pelas equipes de Atenção Básica>,
 u'COBPOP_EAB_11': <Indicador: Cobertura populacional estimada pelas equipes de Atenção Básica>,
 u'COBPOP_EAB_12': <Indicador: Cobertura populacional estimada pelas equipes de Atenção Básica>,
 u'COBPOP_EAB_13': <Indicador: Cobertura populacional estimada pelas equipes de Atenção Básica>,
 u'COBPOP_EAB_14': <Indicador: Cobertura populacional estimada pelas equipes de Atenção Básica>,
 u'CONCL_ENFERM_13': <Indicador: Número de concluintes - cursos de enfermagem>,
 u'CONCL_ENFERM_14': <Indicador: Número de concluintes - cursos de enfermagem>,
 u'CONCL_ENFHAB_13': <Indicador: Concluintes de enfe

In [157]:
for line in lines:
    d = dict(zip([unicode(_) for _ in headers], [_.decode('iso-8859-1') for _ in line]))
    l = regioes[d['REGSAUDCOD_BID']]
    for codigo in [_ for _ in headers[2:] if _ in indicadores]:
        a = codigo_periodo[codigo]
        i = indicadores[codigo]
        try:
            dado = Dado.objects.get(indicador=i, ano=a, localidade=l)
        except Dado.DoesNotExist:
            dado = Dado.objects.create(indicador=i, ano=a, localidade=l, valor=d[codigo])
        else:
            dado.valor = d[codigo]
            dado.save()
    
    

In [159]:
Dado.objects.count()

28791